In [86]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [87]:
#Seven-Day Load Forecast by Weather Zone URL
url = 'http://mis.ercot.com/misapp/GetReports.do?reportTypeId=12312&reportTitle=Seven-Day%20Load%20Forecast%20by%20Weather%20Zone&showHTMLView=&mimicKey'

In [88]:
response = requests.get(url)

In [89]:
soup = bs(response.text, 'html.parser')

In [91]:
title_results = soup.find_all('td', class_='labelOptional_ind')

In [ ]:
for result in title_results:
    print(result.text)

In [93]:
url_results = soup.find_all('td', class_='labelOptional', text=True)

In [ ]:
base_url = 'http://mis.ercot.com'

data_urls = []

for result in url_results:
    try:
        end_url = result.find('div').a['href']
        full_url = base_url + end_url
        data_urls.append(full_url)
        #print(full_url)
    except:
        pass

## Visit URL

In [99]:
data = requests.get(data_urls[0], stream=True)

In [100]:
import requests, zipfile, io

In [101]:
z = zipfile.ZipFile(io.BytesIO(data.content))

In [102]:
z.extractall()

In [103]:
file = z.namelist()[0]

In [184]:
load_fcst = pd.read_csv(file)

In [185]:
load_fcst

,DeliveryDate,HourEnding,Coast,East,FarWest,North,NorthCentral,SouthCentral,Southern,West,SystemTotal,DSTFlag
0,07/25/2020,1:00,12911.0996,1788.4301,3915.1299,1063.8101,15900.7002,8495.8701,4015.9900,1633.4100,49724.4400,N
1,07/25/2020,2:00,12480.7998,1643.0800,3838.8899,1018.4600,14940.5996,7909.5098,3782.8899,1483.2300,47097.4590,N
2,07/25/2020,3:00,12031.5000,1589.1400,3795.8301,981.7140,14000.2002,7495.5498,3569.3799,1388.6801,44851.9941,N
3,07/25/2020,4:00,11798.2998,1557.9200,3735.1499,963.5380,13179.0000,6999.5298,3439.4399,1406.2400,43079.1174,N
4,07/25/2020,5:00,11677.5000,1506.8199,3679.6299,910.7460,12853.5000,6765.1899,3436.1899,1275.6801,42105.2557,N
...,...,...,...,...,...,...,...,...,...,...,...,...
187,08/01/2020,20:00,15860.7998,2173.7000,4259.8999,1383.9399,20614.5996,10854.2998,4844.5801,2004.5200,61996.3391,N
188,08/01/2020,21:00,15946.7002,2116.9199,4229.7700,1353.7900,19982.1992,10489.7998,4751.2202,1961.9800,60832.3793,N
189,08/01/2020,22:00,15535.5000,2071.5500,4218.3301,1303.0601,19329.5996,10049.2002,4697.8301,1907.8500,59112.9201,N
190,08/01/2020,23:00,14774.0000,1932.7900,4086.5500,1209.8600,18012.3008,9414.4004,4475.8799,1802.9000,55708.6811,N


In [186]:
#clean existing columns
load_fcst['DeliveryDate'] = pd.to_datetime(load_fcst.DeliveryDate)
load_fcst['HourEnding'] = load_fcst.HourEnding.str.split(':').str[0]

#add file name and process info to df
load_fcst['FileName'] = z.namelist()[0]
load_fcst['ProcessDate'] = pd.to_datetime(load_fcst.FileName.str.split('.').str[3])
load_fcst['ProcessHour'] = load_fcst.FileName.str.split('.').str[4]
load_fcst['DT'] = pd.to_datetime(load_fcst.FileName.str.split('.').str[3] + load_fcst.FileName.str.split('.').str[4])


In [187]:
load_fcst['DeliveryDT'] = pd.to_datetime(load_fcst.DeliveryDate) + load_fcst.HourEnding.astype('timedelta64[h]')

In [188]:
load_fcst.head()

,DeliveryDate,HourEnding,Coast,East,FarWest,North,NorthCentral,SouthCentral,Southern,West,SystemTotal,DSTFlag,FileName,ProcessDate,ProcessHour,DT,DeliveryDT
0,2020-07-25,1,12911.0996,1788.4301,3915.1299,1063.8101,15900.7002,8495.8701,4015.9900,1633.4100,49724.4400,N,cdr.00012312.0000000000000000.20200725.213002....,2020-07-25,213002,2020-07-25 21:30:02,2020-07-25 01:00:00
1,2020-07-25,2,12480.7998,1643.0800,3838.8899,1018.4600,14940.5996,7909.5098,3782.8899,1483.2300,47097.4590,N,cdr.00012312.0000000000000000.20200725.213002....,2020-07-25,213002,2020-07-25 21:30:02,2020-07-25 02:00:00
2,2020-07-25,3,12031.5000,1589.1400,3795.8301,981.7140,14000.2002,7495.5498,3569.3799,1388.6801,44851.9941,N,cdr.00012312.0000000000000000.20200725.213002....,2020-07-25,213002,2020-07-25 21:30:02,2020-07-25 03:00:00
3,2020-07-25,4,11798.2998,1557.9200,3735.1499,963.5380,13179.0000,6999.5298,3439.4399,1406.2400,43079.1174,N,cdr.00012312.0000000000000000.20200725.213002....,2020-07-25,213002,2020-07-25 21:30:02,2020-07-25 04:00:00
4,2020-07-25,5,11677.5000,1506.8199,3679.6299,910.7460,12853.5000,6765.1899,3436.1899,1275.6801,42105.2557,N,cdr.00012312.0000000000000000.20200725.213002....,2020-07-25,213002,2020-07-25 21:30:02,2020-07-25 05:00:00


In [189]:
load_fcst.describe()

,Coast,East,FarWest,North,NorthCentral,SouthCentral,Southern,West,SystemTotal
count,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000,192.000000
mean,14010.461975,1858.842916,3984.097094,1149.205806,16762.620284,8345.451369,3893.957974,1654.156254,51658.793672
std,1959.017727,311.408995,204.511438,202.891299,3189.946438,1526.269735,621.887156,230.693455,8044.176924
min,10620.599600,1323.640000,3631.590100,832.943000,11643.099600,5735.259800,2858.889900,1194.310100,38242.183400
25%,12220.925050,1572.352500,3786.127575,959.908250,13727.974600,6978.800175,3427.520000,1458.907500,44154.169850
50%,13922.799800,1866.130050,3978.530050,1149.190000,16585.599600,8324.714800,3818.529900,1620.005000,51557.549100
75%,15730.274900,2151.205075,4171.835100,1305.710000,19518.174800,9615.652800,4226.227675,1853.970000,58427.178125
max,17737.500000,2398.450000,4358.390100,1557.660000,23073.699200,11318.099600,5332.549800,2096.530000,67583.549200


In [190]:
load_fcst.dtypes

DeliveryDate    datetime64[ns]
HourEnding              object
Coast                  float64
East                   float64
FarWest                float64
North                  float64
NorthCentral           float64
SouthCentral           float64
Southern               float64
West                   float64
SystemTotal            float64
DSTFlag                 object
FileName                object
ProcessDate     datetime64[ns]
ProcessHour             object
DT              datetime64[ns]
DeliveryDT      datetime64[ns]
dtype: object

In [191]:
z.namelist()[0]

'cdr.00012312.0000000000000000.20200725.213002.LFCWEATHERNP3561.csv'

### Convert DF to Long Format (i.e. Unpivot Forecast columns)

In [208]:
#long_load_fcst = pd.melt(load_fcst, id_vars=['Coast','East','FarWest','North','NorthCentral','SouthCentral','Southern','West','SystemTotal'],var_name='WeatherZone', value_name ='MW')
#long_load_fcst = pd.melt(load_fcst, id_vars='DeliveryDT',var_name='WeatherZone', value_name ='MW')
testdf = load_fcst.stack()

testdf

0    DeliveryDate                                  2020-07-25 00:00:00
     HourEnding                                                      1
     Coast                                                     12911.1
     East                                                      1788.43
     FarWest                                                   3915.13
                                           ...                        
191  FileName        cdr.00012312.0000000000000000.20200725.213002....
     ProcessDate                                   2020-07-25 00:00:00
     ProcessHour                                                213002
     DT                                            2020-07-25 21:30:02
     DeliveryDT                                    2020-08-02 00:00:00
Length: 3264, dtype: object

In [205]:
long_load_fcst.head()

,DeliveryDT,WeatherZone,MW
0,2020-07-25 01:00:00,DeliveryDate,2020-07-25 00:00:00
1,2020-07-25 02:00:00,DeliveryDate,2020-07-25 00:00:00
2,2020-07-25 03:00:00,DeliveryDate,2020-07-25 00:00:00
3,2020-07-25 04:00:00,DeliveryDate,2020-07-25 00:00:00
4,2020-07-25 05:00:00,DeliveryDate,2020-07-25 00:00:00


## Plotting Forecasts

In [192]:
import plotly.express as px

In [193]:
fig = px.line(load_fcst, x="DeliveryDT", y="SystemTotal", title='System Level Forecasts')
fig.show()

In [196]:
fcst_list = [load_fcst.Coast, load_fcst.East, load_fcst.FarWest, load_fcst.North, load_fcst.NorthCentral, load_fcst.SouthCentral, load_fcst.Southern, load_fcst.West]

In [201]:
barfig = px.bar(load_fcst, x='DeliveryDT', y=load_fcst.Coast)
barfig.show()